In [50]:
import numpy as np
import time
import copy

from vgc.datatypes.Objects import PkmTeam, Pkm, GameState, Weather
from vgc.engine.PkmBattleEnv import PkmBattleEnv
from vgc.util.generator.PkmTeamGenerators import RandomTeamGenerator

from vgc.datatypes.Constants import TYPE_CHART_MULTIPLIER, MAX_HIT_POINTS, MOVE_MAX_PP, DEFAULT_TEAM_SIZE
from vgc.datatypes.Objects import PkmMove, Pkm, PkmTeam, GameState, Weather
from vgc.datatypes.Types import PkmStat, PkmType, WeatherCondition, \
    N_TYPES, N_STATUS, N_STATS, N_ENTRY_HAZARD, N_WEATHER, PkmStatus, PkmEntryHazard


import pprint
import pickle

from pkm_battle_env_wrapper import NiBot, SimpleBot
from vgc.behaviour.BattlePolicies import RandomPlayer
import importlib

from scipy.stats import chi2_contingency

In [58]:
import pequil_bot_battle_policy

importlib.reload(pequil_bot_battle_policy)
from pequil_bot_battle_policy import PequilBot

pequil_bot = PequilBot()

In [59]:
num_battles = 30000

RANDOM_BOT_NAME = 'random'
NIBOT_NAME = 'nibot'
SIMPLE_BOT_NAME = 'simple'
PEQUIL_BOT_NAME = 'pequil'

is_pequil_bot_team_1 = True
opponent_bot = SIMPLE_BOT_NAME #NIBOT_NAME#RANDOM_BOT_NAME#NIBOT_NAME



winner_dict = {
    0:0,
    1:0,
    -1:0
}

pkm_env_action_dict = {
    0:{
        0:0,
        1:0,
        2:0,
        3:0,
        4:0,
        5:0,
    },
    1:{
        0:0,
        1:0,
        2:0,
        3:0,
        4:0,
        5:0,
    },
}

action_state_results_dict = {}

max_episode_steps = 250
agent_index = 0


team_generator = RandomTeamGenerator(2)

time_int = int(time.time())
# save_tag =  f"_smoke_test_{time_int}"
start_time = time.time()

pequil_bot = PequilBot()
random_agent = RandomPlayer()
nibot = NiBot()
simple_bot = SimpleBot()

for battle_idx in range(num_battles):
    
    team_1 = team_generator.get_team().get_battle_team([0, 1, 2])
    team_2 = team_generator.get_team().get_battle_team([0, 1, 2])

    

    # set new environment with teams
    env = PkmBattleEnv((team_1, team_2),
                   encode=(False, False)) 

    game_state, info = env.reset()

    is_first_move = True
    agent_first_move = None
    state_key = None

    for episode_step in range(max_episode_steps):
        
        if is_pequil_bot_team_1:
            team_1_action = pequil_bot.get_action(game_state[0])

            if opponent_bot == NIBOT_NAME:
                team_2_action = nibot.get_action(game_state[1])
            elif opponent_bot == SIMPLE_BOT_NAME:
                team_2_action = simple_bot.get_action(game_state[1])
            else:
                team_2_action = random_agent.get_action(game_state[1])
        else:
            if opponent_bot == NIBOT_NAME:
                team_1_action = nibot.get_action(game_state[0])
            elif opponent_bot == SIMPLE_BOT_NAME:
                team_1_action = simple_bot.get_action(game_state[0])
            else:
                team_1_action = random_agent.get_action(game_state[0])

            team_2_action = pequil_bot.get_action(game_state[1])

        # enter action and step the env
        action_list = [team_1_action, team_2_action]
        pkm_env_action_dict[0][team_1_action] += 1
        pkm_env_action_dict[1][team_2_action] += 1

        game_state, _not_used_reward, terminated, truncated, info = env.step(action_list)  # for inference, we don't need reward

        if episode_step == max_episode_steps - 1:
            print('Warning: max steps reached')
            terminated = True

        if terminated:
            winner = env.winner
            winner_dict[winner] += 1
            break


pprint.pprint(winner_dict)
pprint.pprint(pkm_env_action_dict)
# print(action_dict)


{-1: 0, 0: 14944, 1: 15056}
{0: {0: 51304, 1: 31567, 2: 30078, 3: 28586, 4: 286, 5: 151},
 1: {0: 51285, 1: 31579, 2: 30203, 3: 28905, 4: 0, 5: 0}}


In [60]:
def chi_square_test(winner_dict, num_battles):
    contingency_table = [[winner_dict[0], winner_dict[1]], [winner_dict[1], winner_dict[0]]]
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    print(f'Chi-square statistic: {chi2:.3f}')
    print(f'P-value: {p_value:.5f}')
    print(f'team_1 win rate: {winner_dict[0] / num_battles:.3f}')
    print(f'team_2 win rate: {winner_dict[1] / num_battles:.3f}')

chi_square_test(winner_dict, num_battles)


Chi-square statistic: 0.821
P-value: 0.36477
team_1 win rate: 0.498
team_2 win rate: 0.502
